In [18]:
import pandas as pd
import  os
import numpy as np
import psycopg2
import glob
import openpyxl

In [31]:
df = pd.read_csv("Journal.csv")
df.head()

,Journal Number,Journal Number Prefix,Journal Number Suffix,Journal Date,Tax Basis,Status,Journal Type,Reference Number,Notes,Is Inclusive Tax,...,Tax Type,Project Name,Debit,Credit,Account,Account Code,Contact Name,Currency,Total,Description
0,1,NaN,1,2018-11-01,Both,Published,Journal,NaN,Capital contribution for various expenses,False,...,Amount,NaN,50000.00,0.00,Owner's Equity,124000002,NaN,GHS,50000.00,Capital contribution for various expenses
1,1,NaN,1,2018-11-01,Both,Published,Journal,NaN,Capital contribution for various expenses,False,...,Amount,NaN,0.00,50000.00,Stanbic - GHS,111000007,NaN,GHS,50000.00,Capital contribution for various expenses
2,2,NaN,2,2021-01-01,Both,Published,Journal,NaN,Opening balance from 2020 financial statement,False,...,Amount,NaN,0.00,35616.53,Opening Balance Adjustments,121000004,NaN,GHS,35616.53,NaN
3,2,NaN,2,2021-01-01,Both,Published,Journal,NaN,Opening balance from 2020 financial statement,False,...,Amount,NaN,35616.53,0.00,Directors account,124000005,NaN,GHS,35616.53,NaN
4,11,NaN,11,2021-09-21,Both,Published,Journal,Afrigeo Trade Finance,$270k cargo financing over 60 days to purchase...,False,...,Amount,NaN,270000.00,0.00,Trade Finance Loan,112000004,Afrigeo Ghana Limited - USD,USD,270000.00,$270k cargo financing over 60 days to purchase...


In [20]:
df.columns = [x.lower().replace("#","number").replace(" ","_").replace("%","").replace(".","") for x in df.columns]
df.columns

Index(['journal_number', 'journal_number_prefix', 'journal_number_suffix',
       'journal_date', 'tax_basis', 'status', 'journal_type',
       'reference_number', 'notes', 'is_inclusive_tax', 'exchange_rate',
       'branch_id', 'branch_name', 'item_order', 'tax_name', 'tax_percentage',
       'tax_amount', 'tax_type', 'project_name', 'debit', 'credit', 'account',
       'account_code', 'contact_name', 'currency', 'total', 'description'],
      dtype='object')

In [21]:
df.dtypes

journal_number             int64
journal_number_prefix    float64
journal_number_suffix      int64
journal_date              object
tax_basis                 object
status                    object
journal_type              object
reference_number          object
notes                     object
is_inclusive_tax            bool
exchange_rate            float64
branch_id                float64
branch_name               object
item_order                 int64
tax_name                 float64
tax_percentage           float64
tax_amount               float64
tax_type                  object
project_name             float64
debit                    float64
credit                   float64
account                   object
account_code               int64
contact_name              object
currency                  object
total                    float64
description               object
dtype: object

In [22]:
replacements = {
    'object' : 'varchar',
    'float64' : 'float',
    'int64' : 'float',
    'datetime64[ns]': 'varchar',
    'datetime64': 'timestamp',
    'bool' : 'varchar'
}
replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'float',
 'datetime64[ns]': 'varchar',
 'datetime64': 'timestamp',
 'bool': 'varchar'}

In [23]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'journal_number float, journal_number_prefix float, journal_number_suffix float, journal_date varchar, tax_basis varchar, status varchar, journal_type varchar, reference_number varchar, notes varchar, is_inclusive_tax varchar, exchange_rate float, branch_id float, branch_name varchar, item_order float, tax_name float, tax_percentage float, tax_amount float, tax_type varchar, project_name float, debit float, credit float, account varchar, account_code float, contact_name varchar, currency varchar, total float, description varchar'

In [24]:
conn_string = "host= db.app.production.jetstreamafrica.com \
                dbname='jetstream-finance' \
                    user='ujetstreamfinance' password='Smehr5A7FGw4jEX@' "

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

print('opened database successfully')


opened database successfully


In [25]:
cursor.execute("drop table if exists journals;")

In [26]:
cursor.execute("create table journals \
    (journal_number float, journal_number_prefix float, journal_number_suffix float, journal_date varchar, tax_basis varchar, status varchar, journal_type varchar, reference_number varchar, notes varchar, is_inclusive_tax varchar, exchange_rate float, branch_id float, branch_name varchar, item_order float, tax_name float, tax_percentage float, tax_amount float, tax_type varchar, project_name float, debit float, credit float, account varchar, account_code float, contact_name varchar, currency varchar, total float, description varchar)")

In [27]:
#insert values to tables
#save df to csv
df.to_csv('journals', header=df.columns, index=False, encoding='utf-8')
#open the csv file, save it as an object and upload to db
my_file = open("journals")
print('file opened in memory')

file opened in memory


In [28]:
SQL_STATEMENT = """
COPY journals FROM STDIN WITH
   CSV
   HEADER
   DELIMITER AS ','
"""
cursor.copy_expert(sql = SQL_STATEMENT, file=my_file)
print('file copied to db')

file copied to db


In [29]:
cursor.execute("grant select on table journals to public")
conn.commit()

cursor.close()
print('table journals made imported successfully')

table journals made imported successfully
